In [9]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [13]:
import re
import string
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize
from nltk.corpus import wordnet

# 🔁 Download required NLTK data if not already present
nltk_packages = ['punkt', 'stopwords', 'wordnet', 'omw-1.4']
for pkg in nltk_packages:
    try:
        nltk.data.find(f'corpora/{pkg}')
    except LookupError:
        nltk.download(pkg)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/shubhamagarwal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/shubhamagarwal/nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/shubhamagarwal/nltk_data...


In [10]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
183,I really wanted to love this film. I have read...,negative
46,"My parents may enjoy this show, but I fail to ...",negative
774,About the baby: Why wasn't big brother assumin...,negative
327,This has to be the worst movie i've seen this ...,negative
616,My husband and I just got done watching this m...,positive


In [11]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [15]:
df = normalize_text(df)
df.head()

,review,sentiment
183,really wanted love film read book daughter lov...,negative
46,parent may enjoy show fail find humor funny de...,negative
774,baby big brother assuming hungry bottle nouris...,negative
327,worst movie seen year watch lot cable plot rid...,negative
616,husband got done watching movie expecting good...,positive


In [16]:
df['sentiment'].value_counts()

sentiment
negative    270
positive    230
Name: count, dtype: int64

In [17]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [18]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
183,really wanted love film read book daughter lov...,0
46,parent may enjoy show fail find humor funny de...,0
774,baby big brother assuming hungry bottle nouris...,0
327,worst movie seen year watch lot cable plot rid...,0
616,husband got done watching movie expecting good...,1


In [19]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [20]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [22]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/shubhamagr24/Sentiment_Clasification_Complete_MLOPS.mlflow')
dagshub.init(repo_owner='shubhamagr24', repo_name='Sentiment_Clasification_Complete_MLOPS', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

/opt/anaconda3/envs/atlas/lib/python3.10/site-packages/rich/live.py:229: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=a2941e81-847c-4147-a512-ca58e7286db3&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=10f3bc077912c6077d73bb2271e613ca7bb70f50abb17eb316700f5e7048a0c1




Accessing as shubhamagr24

Initialized MLflow to track repo "shubhamagr24/Sentiment_Clasification_Complete_MLOPS"

Repository shubhamagr24/Sentiment_Clasification_Complete_MLOPS initialized!

2025/04/05 23:02:14 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/38a1e13d9dc040f9874e9f0a8d668b06', creation_time=1743874335019, experiment_id='0', last_update_time=1743874335019, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [23]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-04-05 23:03:01,373 - INFO - Starting MLflow run...
2025-04-05 23:03:01,882 - INFO - Logging preprocessing parameters...
2025-04-05 23:03:03,142 - INFO - Initializing Logistic Regression model...
2025-04-05 23:03:03,142 - INFO - Fitting the model...
2025-04-05 23:03:03,261 - INFO - Model training complete.
2025-04-05 23:03:03,261 - INFO - Logging model parameters...
2025-04-05 23:03:03,660 - INFO - Making predictions...
2025-04-05 23:03:03,661 - INFO - Calculating evaluation metrics...
2025-04-05 23:03:03,669 - INFO - Logging evaluation metrics...
2025-04-05 23:03:05,164 - INFO - Saving and logging the model...
2025/04/05 23:03:11 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2025-04-05 23:03:11,544 - INFO - Model training and logging completed in 9.66 seconds.
2025-04-05 23:03:11,545 - INFO - Accuracy: 0.672
2025-04-05 23:03:11,545 - INFO - Precision: 0.5555555555555556
2025-04-